In [2]:
import os, sys, glob, random
import numpy as np
import pandas as pd
import SimpleITK as sitk
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, Pool
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, roc_auc_score, average_precision_score, confusion_matrix, brier_score_loss
from hyperopt import hp, tpe, Trials
from hyperopt.fmin import fmin
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

import compare_auc_delong_xu

np.random.seed(529)
scaler = StandardScaler()

In [3]:
def getstats(y_true,y_pred,y_prob):
    rocauc = roc_auc_score(y_true,y_prob[:,1])
    brsc = round(brier_score_loss(y_true, y_prob[:,1]),2)
    prauc = round(average_precision_score(y_true,y_prob[:,1]),2)
    TN, FP, FN, TP = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    pos_pred_val = round(TP/ (TP+FP),2)
    neg_pred_val = round(TN/ (TN+FN),2)
    stats = [rocauc, prauc, brsc, pos_pred_val, neg_pred_val]
    return stats

def getAUC_CI(labels,scores):
    alpha = 0.95
    aucs = roc_auc_score(labels,scores)
    auc_delong, variances = compare_auc_delong_xu.delong_roc_variance(labels,scores)
    auc_std = np.sqrt(variances)
    lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)
    ci = stats.norm.ppf(lower_upper_q,loc=auc_delong,scale=auc_std)
    return auc_delong, variances, ci

In [4]:
basefile = pd.read_csv('base_data_file.csv',low_memory=False)
# merging the base file with radiomics feature file
df1 = pd.read_csv('/data/project/bhattlab/appr/PyRadiomics_features_set/Phase3/phase3_data.csv')
df1base = pd.merge(df1,basefile,on='sid')
print('Size after merging with radiomics feature file',df1base.shape)
print('Class Distribution\n',df1base['COPD_P3_x'].value_counts())
#saving the file for Table 1 and Table 3 measurements
df1.to_csv('low-dose-dataframe-radiology.csv',index=False)
#dropping the subject id column and label column
X1 = df1base.iloc[:,1:124].astype('float64')
y1 = df1base['COPD_P3_x']

In [5]:
seed = 529
Xt, Xv, yt, yv = train_test_split(X1, y1, test_size = 0.2, random_state = seed, stratify=y1)

In [6]:
Xt, Xv = np.array(Xt), np.array(Xv)
#splitting the data into demographics (first 6 columns), ct emphysema (7 column), and radiomics
Xtdemo, Xtqct, Xtrad = Xt[:,0:6], Xt[:,6:7], Xt[:,8:]
Xvdemo, Xvqct, Xvrad = Xv[:,0:6], Xv[:,6:7], Xv[:,8:]
Xtrad, Xvrad = StandardScaler().fit_transform(Xtrad), StandardScaler().fit_transform(Xvrad)
Xtcomb = np.concatenate((Xtdemo,Xtqct,Xtrad),axis=1)
Xvcomb = np.concatenate((Xvdemo,Xvqct,Xvrad),axis=1)

clfdemo = CatBoostClassifier(loss_function='CrossEntropy')
clfdemo.fit(Xtdemo,yt)
y_pred_demo = clfdemo.predict(Xvdemo)
y_prob_demo = clfdemo.predict_proba(Xvdemo)
demostats = getstats(yv,y_pred_demo,y_prob_demo)

clfqct = CatBoostClassifier(loss_function='CrossEntropy')
clfqct.fit(Xtqct,yt)
y_pred_qct = clfqct.predict(Xvqct)
y_prob_qct = clfqct.predict_proba(Xvqct)
qctstats = getstats(yv,y_pred_qct,y_prob_qct)

clfrad = CatBoostClassifier(loss_function='CrossEntropy')
clfrad.fit(Xtrad,yt)
y_pred_rad = clfrad.predict(Xvrad)
y_prob_rad = clfrad.predict_proba(Xvrad)
radstats = getstats(yv,y_pred_rad,y_prob_rad)

clfcomb = CatBoostClassifier(loss_function='CrossEntropy')
clfcomb.fit(Xtcomb,yt)
y_pred_comb = clfcomb.predict(Xvcomb)
y_prob_comb = clfcomb.predict_proba(Xvcomb)
combstats = getstats(yv,y_pred_comb,y_prob_comb)

In [7]:
aucdemo, aucvardemo, auccidemo = getAUC_CI(yv,y_prob_demo[:,1])
aucqct, aucvarqct, aucciqct = getAUC_CI(yv,y_prob_qct[:,1])
aucrad, aucvarrad, auccirad = getAUC_CI(yv,y_prob_rad[:,1])
auccomb, aucvarcomb, auccicomb = getAUC_CI(yv,y_prob_comb[:,1])

print('demographics -> CI ',aucdemo,auccidemo)
print('CT emphysema -> CI ',aucqct,aucciqct)
print('CT radiomics -> CI ',aucrad, auccirad)
print('combined     => CI ',auccomb, auccicomb)

auc_rad_vs_demo = compare_auc_delong_xu.delong_roc_test(yv,y_prob_rad[:,1],y_prob_demo[:,1])
auc_rad_vs_qct = compare_auc_delong_xu.delong_roc_test(yv,y_prob_rad[:,1],y_prob_qct[:,1])
auc_rad_vs_comb = compare_auc_delong_xu.delong_roc_test(yv,y_prob_rad[:,1],y_prob_comb[:,1])
print('AUC Comparison - Demographics vs. Radiomics, DeLong P-value ', np.exp(auc_rad_vs_demo))
print('AUC Comparison - CT emphysema vs. Radiomics, DeLong P-value ',np.exp(auc_rad_vs_qct))
print('AUC Comparison - Combined vs. Radiomics, DeLong P-value ',np.exp(auc_rad_vs_comb))


lowdose_feature_imp = pd.DataFrame(sorted(zip(clfrad.feature_importances_,X1.columns[8:])), columns=['Value','Feature'])
lowdose_feature_imp = lowdose_feature_imp.sort_values(by="Value", ascending=False)
lowdose_feature_imp[0:10]

demographics -> CI  0.7000461631334116 [0.64498966 0.75510266]
CT emphysema -> CI  0.7466531728276695 [0.69485187 0.79845448]
CT radiomics -> CI  0.8718085295266504 [0.83324468 0.91037238]
combined     => CI  0.8888178686836405 [0.85465077 0.92298497]
AUC Comparison - Demographics vs. Radiomics, DeLong P-value  [[0.00115041]]
AUC Comparison - CT emphysema vs. Radiomics, DeLong P-value  [[0.002989]]
AUC Comparison - Combined vs. Radiomics, DeLong P-value  [[0.32430733]]


,Value,Feature
114,5.517764,original_shape_Sphericity
113,4.205237,rightoriginal_shape_Sphericity
112,3.229163,original_glszm_GrayLevelNonUniformityNormalized
111,2.846843,original_firstorder_10Percentile
110,2.494523,leftoriginal_shape_Sphericity
109,2.368065,original_glrlm_LongRunLowGrayLevelEmphasis
108,2.304361,original_shape_SurfaceVolumeRatio
107,2.294795,original_glrlm_RunEntropy
106,1.995239,rightoriginal_shape_Flatness
105,1.820820,original_shape_LeastAxisLength
